## The Stableswap equation

The Stableswap equation given in the Curve whitepaper is

$$ An^n\sum_i x_i + D = An^n D + \frac{D^{n+1}}{n^n\prod\limits_i x_i}$$

$D$ is the stableswap invariant.

$A$ is the amplification coefficient.

The $x_i$'s are the pool balances for each token.

## Solving for $D$ using Newton's method

To solve for $D$ in the stableswap equation, we can use the auxiliary form of the equation

$$ f(D) = 0 $$

where

$$ f(D) = An^n D + \frac{D^{n+1}}{n^n\prod x_i} - An^n S - D $$

Note $f(P) < 0$ and $f(S) > 0$ (this is simple to see using that $P <= S$ with equality only when $x_1 = x_2$).  Since $P < S$ in the generic case, we expect that somewhere in-between, there is a $D$ such that $f(D) = 0$.  In fact, the situation is much better than that.  

The derivative of $f$ is

$$ f'(D) = An^n + (n+1) \frac{D^n}{n^n\prod x_i} - 1 $$

Since $f'$ is always be negative (as long as $A > 1$), $f$ is strictly decreasing and Newton's method will rapidly find a solution.

Newton's method gives the iteration:

$$ x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)} $$

After some cleanup, this gives the iterative formula:

$$ D = \frac{(nD_p + An^nS)D}{(n+1)D_p + (An^n - 1)D} $$

where $D_p = \frac{D^{n+1}}{n^n\prod x_i}$

## Convergence under integer arithmetic

Due to the necessity of implementing the above iterative formula in integer arithmetic for the EVM, some trickiness arises.

First, note that 

### Calculation in vyper (from 3Pool)

```python
def get_D(xp: uint256[N_COINS], amp: uint256) -> uint256:
    S: uint256 = 0
    for _x in xp:
        S += _x
    if S == 0:
        return 0

    Dprev: uint256 = 0
    D: uint256 = S
    Ann: uint256 = amp * N_COINS
    for _i in range(255):
        D_P: uint256 = D
        for _x in xp:
            D_P = D_P * D / (_x * N_COINS)
        Dprev = D
        D = (Ann * S + D_P * N_COINS) * D / ((Ann - 1) * D + (N_COINS + 1) * D_P)
        if D > Dprev:
            if D - Dprev <= 1:
                break
        else:
            if Dprev - D <= 1:
                break
    return D
```

In [113]:
from math import prod

def f(D, xp):
    """Useful to check accuracy of solution."""
    S = sum(xp)
    n = len(xp)
    Ann = A * n**n
    return Ann * D + D**(n+1) / (n**n * prod(xp)) - Ann * S - D
    

In [114]:
# this causes the prod ("real") calc to bork
A = 100
x_1 = 98_500_000 * 10**18
x_2 = 5 * 10**18

In [118]:
A = 100
x_1 = 98_500_000 * 10**18
x_2 = 500_000 * 10**18

In [119]:
# iterative calculation for D invariant

xp = [x_1, x_2]

n = len(xp)
S = sum(xp)


Ann = A * n**n

## Commented-out since this can enter an infinite loop!
# D = S
# Dprev = 0
# while abs(Dprev - D) > 1:
#     print("D:", D, "diff:", Dprev - D)
#     D_P = D
#     for x in xp:
#         D_P = D_P * D // (x * n)
#     Dprev = D
#     D = (Ann * S + D_P * n) * D // ((Ann - 1) * D + (n + 1) * D_P)
#     if Dprev < D:
#         print("bounce", Dprev, D)
# print("Solution:", D, "diff:", Dprev - D)
# print("f(D):", f(D, xp))

print("")
print("Integer version with 'bounce' stop condition:\n")

D = S
Dprev = S + 1
while Dprev > D:
    print("D:", D, "diff:", Dprev - D)
    D_P = D
    for x in xp:
        D_P = D_P * D // (x * n)
    Dprev = D
    D = (Ann * S + D_P * n) * D // ((Ann - 1) * D + (n + 1) * D_P)
print("D:", D, "diff:", Dprev - D)
print("Solution:", Dprev)
print("f(D):", f(Dprev, xp))

print("")
print("Floating point version:\n")

D = S
Dprev = S + 1
while Dprev > D:
    print("D:", D, "diff:", Dprev - D)
    D_P = D**(n+1) / (prod(xp) * n**n)
    Dprev = D
    D = (Ann * S + D_P * n) * D / ((Ann - 1) * D + (n + 1) * D_P)
    D = int(D)
print("D:", D, "diff:", Dprev - D)
print("Solution:", Dprev)
print("f(D):", f(Dprev, xp))




Integer version with 'bounce' stop condition:

D: 99000000000000000000000000 diff: 1
D: 90196822398755624687517360 diff: 8803177601244375312482640
D: 89980006434373679281203737 diff: 216815964381945406313623
D: 89979882906659187435891502 diff: 123527714491845312235
D: 89979882906619154845147287 diff: 40032590744215
D: 89979882906619154845147283 diff: 4
D: 89979882906619154845147283 diff: 0
Solution: 89979882906619154845147283
f(D): 1546188226560.0

Floating point version:

D: 99000000000000000000000000 diff: 1
D: 90196822398755619163078656 diff: 8803177601244380836921344
D: 89980006434373685871640576 diff: 216815964381933291438080
D: 89979882906659186414714880 diff: 123527714499456925696
D: 89979882906619140139646976 diff: 40046275067904
D: 89979882906619157319516160 diff: -17179869184
Solution: 89979882906619140139646976
f(D): -7232724926464.0
